### Opakování &ndash; užitečné věci z Pythonu

In [4]:
my_string = "to be Or NOT to be"
my_list = my_string.split()
print 'Delka stringu je:', len(my_string)
print 'String obsahuje slova:', my_string.split()
print 'String obsahuje unikatni slova:', set(my_string.split())
print 'String prevedeny na mala pismena: ', my_string.lower()
print 'Delka pole (pocet slov) je:', len(my_list)
print 'Prvni prvek v poli je:', my_list[0]
print 'Posledni prvek v poli je:', my_list[-1]

Delka stringu je: 18
String obsahuje slova: ['to', 'be', 'Or', 'NOT', 'to', 'be']
String obsahuje unikatni slova: {'be', 'Or', 'NOT', 'to'}
String prevedeny na mala pismena:  to be or not to be
Delka pole (pocet slov) je: 6
Prvni prvek v poli je: to
Posledni prvek v poli je: be


### Spuštění interaktivního shellu PySpark

`pyspark --master yarn --num-executors 2 --executor-memory 4G --conf spark.ui.port=1<ddmm>`, kde `<ddmm>` je váš den a měsíc narození, např. `spark.ui.port=10811`

### Úkol č. 1: rozběhání příkladu z přednášky (word count)

Vyzkoušejte, že příklad z přednášky funguje.

In [ ]:
# nacteni RDD
lines = sc.textFile("/user/pascepet/data/bible/bible.txt")

# rozdeleni radku
words = lines.flatMap(lambda line: line.split(" "))

# transformace radku na (key, value)
pairs = words.map(lambda word: (word, 1))

# secteni jednicek ke kazdemu klici
counts = pairs.reduceByKey(lambda a, b: a + b)

# vypis vysledku
counts.take(5)

### Úkol č. 2: vylepšení příkladu z přednášky

Vylepšete skript z úkolu č. 1 o následující funkcionality:

2.1. Ignorujte začátky řádků (název biblické knihy a kód kapitola:verš &ndash; jsou odděleny od textu tabulátorem).  
2.2. Slova, která se liší jen velikostí písmen, se budou považovat za stejná.  
2.3. Odstraňte z textu nealfanumerické znaky &ndash; všechny nebo aspoň některé z nich (např. '.', ':', '-' atd.).  
2.4. Vyřaďte ze zpracování slova v množině tzv. *stopwords* (v souboru `/user/pascepet/data/stopwords.txt`).   
*Hint: stopwords můžete dostat do proměnné typu set např. takto:*  
`sw = set(sc.textFile('/user/pascepet/data/stopwords.txt').collect())`  
2.5. Vyberte RDD, které je vhodné si pro další výpočty kešovat, a nakešujte ho.  
2.6. Na konci seřaďte slova podle četnosti sestupně (použijte [metodu sortBy](https://spark.apache.org/docs/1.6.0/api/python/pyspark.html)).  
2.7. Najděte nejdelší slovo (s největším počtem znaků).

In [ ]:
# package re kvuli pouziti regularniho vyrazu
import re

# nacteni RDD
lines = sc.textFile("/user/pascepet/data/bible/bible.txt")

# vzit cast radku po tabulatoru (zacatek radku se zahodi)
lines2 = lines.map(lambda line: line.split("\t")[1])

# rozdeleni radku
words = lines2.flatMap(lambda line: line.split(" "))

# prevedeni slov na mala pismena
# vypusteni nealfanumerickych znaku
# ponechani jen slov, ktera nejsou stopwords
sw = set(sc.textFile('/user/pascepet/data/stopwords.txt').collect())
words2 = words.map(lambda word: word.lower()) \
    .map(lambda word: re.sub(r'\W+', '', word)) \
    .filter(lambda word: word not in sw)

# poznamka: odstraneni jen urcitych znaku lze provest napr. takto
# words2 = words2.map(lambda word: word.replace('.', '')

# nakesovani
words2.cache()

# transformace radku na (key, value)
pairs = words2.map(lambda word: (word, 1))

# secteni jednicek ke kazdemu klici
counts = pairs.reduceByKey(lambda a, b: a + b)

# serazeni podle cetnosti
counts_sorted = counts.sortBy(lambda item: item[1], ascending=False)

# vypis vysledku
counts_sorted.take(20)

# vypocet nejdelsiho slova - prirazeni delky ke slovu
words_len = words2.distinct() \
    .map(lambda word: (word, len(word)))

# setrideni slov podle delky
words_len_sorted = words_len.sortBy(lambda w: w[1], ascending=False)
words_len_sorted.take(5)

### Úkol č. 3: další analýza textu

V této části budeme pracovat s původním textem.

3.1. Zjistěte počet slov v každém verši (1 verš = 1 řádek) a najděte verše s největšim a nejmenším počtem slov.  
3.2. Proveďte stejný výpočet jako v 3.1, ale v každém verši počítejte jen unikátní slova (každé slovo jen jednou).

In [ ]:
# nejdelsi a nejkratsi vers
# oddeli se zacatek radku a zbytek
line_split = lines.map(lambda line: line.split("\t"))

# ze zbytku radku se zjisti pocet slov a radky se podle toho seradi
lines_len = line_split.map(lambda line: (line[0], len(line[1].split(" ")))) \
    .sortBy(lambda vers: vers[1], ascending=False)

lines_len.take(1)
lines_len.sortBy(lambda vers: vers[1], ascending=True).take(1)

# nejdelsi a nejkratsi vers s unikatnimi slovy
# tentokrat se ze zbytku radku nejdriv vezmou jen unikatni slova a az ta se spocitaji
lines_len_uniq = line_split.map(lambda line: (line[0], len(set(line[1].split(" "))))) \
    .sortBy(lambda vers: vers[1], ascending=False)

lines_len_uniq.take(1)
lines_len_uniq.sortBy(lambda vers: vers[1], ascending=True).take(1)

### Úkol č. 4: číselné výpočty z dat

#### Data

Budeme pracovat se soubory hodinových údajů o teplotách (použili jsme ho na supercvičení Hive). Pokud jste na tomto supercvičení byli, pravděpodobně už máte data na HDFS (v podadresáři svého pracovního adresáře, jsou to soubory `teplota1-6.csv` a `teplota7-12.csv`) Zkontrolujte to.  
Nemáte-li je tam, proveďte následující kroky:

* zkopírujte soubor `/home/pascepet/fel_bigdata/data/teplota-usa.zip` do svého uživatelského adresáře (doporučuji si na to vytvořit podadresář);
* rozbalte;
* vytvořte na HDFS ve svém uživatelském adresáři vhodný podadresář;
* rozbalené soubory nakopírujte na HDFS do nově vytvořeného adresáře.

CSV soubor má jako oddělovač znak `','`. Také obsahuje hlavičky s názvy sloupců, které je potřeba při zpracování odstranit.    
Teplota je uvedena v 10&times;&deg;F. Některé řádky obsahují na místě teploty prázdný řetězec, tj. údaj pro ně není k dispozici.

**Sloupce:** id_stanice, mesic, den, hodina, teplota, flag, latitude, longitude, vyska, stat, nazev

*Poznámka: Spark umí v metodě textFile načíst celý adresář (spojí všechny soubory v adresáři do jednoho RDD).*

#### Zadání

4.1. Prohlédněte si několik prvních řádků z některého souboru, abyste si připomněli, co v datech je.  
4.2. Zjistěte, který stát ma nejvyšší průměrnou teplotu z měření jen za měsíce 6&ndash;8. Teplotu uveďte ve stupních Celsia.  
4.3. Pro každý měsíc vypište stát, který má nejvyšší průměrnou teplotu z měření za tento měsíc.


#### Očekávaný výstup

V zadání 4.2

| stat | prum_teplota |  
| ---- | ------------:|  

V zadání 4.3

| mesic | stat | prum_teplota |  
| ----- | ---- | ------------:|  


In [ ]:
# nacteni RDD, odfiltrovani nezadoucich radku
# !!! zde je treba dosadit cestu do vlastniho user adresare na HDFS
lines = sc.textFile("/user/pascepet/data/teplota")

# vyhozeni radku s nazvy sloupcu a ponechani ostatnich
lines2 = lines.filter(lambda line: not(re.match(r'stanice', line)))

# prevod radku na strukturu ((mesic, stat), teplota):
# rozdeleni radku
# ponechani jen radku, kde paty prvek (teplota) je neprazdny
# konverze z Fahrenheita na Celsia
recs = lines2.map(lambda line: line.split(",")) \
    .filter(lambda rec: rec[4]!='') \
    .map(lambda rec: ((int(rec[1]), rec[9]), (float(rec[4])/10-32)*5/9))

# nakesovani
recs.persist(StorageLevel.MEMORY_AND_DISK)

# stat s nejvetsi prumernou teplotou za mesice 6-8
# vyhodit udaje za jine mesice
recs1 = recs.filter(lambda rec: rec[0][0] in range(6,9))
# ponecha jen nazev statu (mesic nas uz nezajima), k tomu pripravi dvojice pro agregaci
recs1 = recs1.map(lambda rec: (rec[0][1], (1, rec[1])))
result1 = recs1.reduceByKey(lambda s,t: (s[0]+t[0], s[1]+t[1]))  # agregace po statech - pocet a soucet 
result1 = result1.map(lambda res: (res[0], res[1][1]/res[1][0])) # prumer = soucet/pocet
result1 = result1.sortBy(lambda res: res[1], False)
result1.take(1)

# za kazdy mesic stat s nejvetsi prumernou teplotou
# spocita se agregace za kazdou dvojici (mesic, stat)
recs2 = recs.map(lambda rec: (rec[0], (1, rec[1])))
result2 = recs2.reduceByKey(lambda s,t: (s[0]+t[0], s[1]+t[1]))
result2 = result2.map(lambda res: (res[0], res[1][1]/res[1][0])) # nyni mame za kazdou dvojici (mesic, stat) prumernou teplotu
result2 = result2.map(lambda res: (res[0][0], (res[0][1], res[1]))) # dame mesic jako klic a mezi staty budeme hledat maximum 
result3 = result2.reduceByKey(lambda s,t: (t if t[1]>s[1] else s))
result3 = result3.sortBy(lambda res: res[0], True)
result3.collect()